In [1]:
import numpy as np

with open('input.txt', 'r') as file:
    data = file.read()
    data = data.split('\n')
    
count_tasks = int(data[0])
difficult = np.array(list(map(int, data[1].split())))
timeh = np.array(list(map(float, data[2].split())))
count_empoyee = int(data[3])
params = np.array([list(map(float, i.split())) for i in data[4:]])
    
print(count_tasks, 
      difficult,
      timeh, 
      count_empoyee,
      params,
      sep='\n')

#print(np.random.randint(low=1, high=count_empoyee+1, size=[2, count_tasks], dtype=int))

1000
[2 3 1 2 4 4 2 2 2 4 3 3 4 4 1 3 2 4 4 1 3 1 2 2 3 2 3 2 4 4 4 3 2 3 4 4 3
 1 3 4 1 1 2 2 3 2 1 3 4 3 2 2 4 2 2 1 2 3 2 3 2 4 3 2 3 1 3 2 1 4 2 4 4 4
 3 2 2 1 4 1 2 4 2 1 4 1 2 2 4 3 2 3 3 1 1 3 2 1 4 4 4 4 1 2 2 1 4 1 3 2 3
 1 4 4 3 3 1 1 4 1 4 1 1 2 2 4 1 3 3 3 3 4 1 4 2 4 2 1 1 2 1 3 2 4 3 4 2 4
 3 1 3 4 2 4 3 2 3 2 3 4 1 4 3 4 2 2 3 3 4 4 2 4 2 4 2 3 1 4 1 1 2 4 2 2 4
 4 3 3 4 2 2 2 1 4 3 1 3 3 4 3 2 1 4 3 3 1 4 3 3 4 1 3 4 2 1 3 1 2 3 2 2 2
 3 4 1 2 1 1 4 1 2 2 4 2 4 4 4 1 4 3 1 3 1 4 1 3 1 4 2 1 2 2 1 2 2 1 1 3 2
 1 2 3 1 2 4 1 3 4 4 1 3 2 1 2 4 4 1 1 3 4 3 4 2 4 4 2 1 2 1 2 3 2 3 3 1 2
 2 3 2 2 3 1 1 1 4 3 1 1 3 1 1 1 2 4 2 4 4 1 1 1 1 2 3 1 4 4 3 2 2 3 1 3 2
 4 1 1 2 1 1 1 1 1 1 4 2 4 3 4 4 3 4 1 1 2 4 4 2 3 4 4 2 4 1 4 2 4 3 4 2 4
 3 3 1 2 3 1 2 2 1 4 2 4 1 3 1 1 2 3 2 4 4 3 4 3 4 2 2 2 2 3 4 2 2 4 4 2 2
 4 3 1 2 1 1 2 4 2 3 4 1 4 3 2 4 4 1 4 3 4 2 1 2 2 1 2 3 4 1 4 1 2 4 1 4 1
 1 2 1 4 1 2 4 3 3 3 1 1 3 4 2 4 2 2 4 2 4 3 1 3 3 4 2 2 2 2 2 4 3 4 1 1 2
 4 1 3 4 2 1 3 2 4 2

In [2]:
task_time = np.array(list(zip(difficult, timeh)))
task_time

array([[2. , 4.5],
       [3. , 9. ],
       [1. , 8.5],
       ...,
       [3. , 5. ],
       [2. , 6. ],
       [1. , 2.5]])

In [3]:
class Genetic:
    
    def __init__(self, len_chromosome, count_indiv, gen_low, gen_high,
                 employee_params, task_diff, task_time) -> None:
        self._individs = np.random.randint(low=gen_low, high=gen_high, size=[count_indiv, len_chromosome], dtype=int)
        self._params = employee_params
        self._diff = task_diff-1
        self._time = task_time
        self.len = len_chromosome
    
    def get_individs(self) -> np.ndarray:
        return self._individs
    
    def score(self, individs: np.ndarray) -> np.ndarray:
        individ_scores = np.array([])
        
        if len(individs.shape) != 1:
        
            for ind in individs:
                indiv_params_full = self._params[ind]
                ind_params = indiv_params_full[np.arange(0, self.len), self._diff]
                total_indiv_val = ind_params * self._time
                indiv_val = np.bincount(ind, weights=total_indiv_val)
                individ_scores = np.append(individ_scores, 1_000_000 / np.max(indiv_val))
        else:
            indiv_params_full = self._params[individs]
            ind_params = indiv_params_full[np.arange(0, self.len), self._diff]
            total_indiv_val = ind_params * self._time
            indiv_val = np.bincount(individs, weights=total_indiv_val)
            individ_scores = np.append(individ_scores, 1_000_000 / np.max(indiv_val))
            
        return individ_scores
    
    def selection(self, fitness: np.ndarray, n: int) -> list:
        
        total_fitness = np.sum(fitness)
        dist = total_fitness/n
        
        start = np.random.ranf() * dist
        
        pointers = [start + i*dist for i in range(n)]
        fit_sums = [np.sum(fitness[:i]) for i in range(1, len(fitness)+1)]
            
        keep = list()

        search_idx = 0
        for p in pointers:
            i = search_idx
            while fit_sums[i] < p:
                i += 1
            keep.append(i)
            search_idx = i
                
        return keep
    
    def evolution_k_point(self, a: np.ndarray, b: np.ndarray, points: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    
        A, B = np.copy(a), np.copy(b)
        
        full_points = np.insert([0, len(a)], 1, points)
        pair_points = list(zip(full_points[:-1], full_points[1:]))

        for i, val in enumerate(pair_points):
            if i % 2:
                A[ val[0]+1:val[1] ], B[ val[0]+1:val[1] ] = b[ val[0]+1:val[1] ], a[ val[0]+1:val[1] ]
                
        return (A,B)
        
    def solution(self, generations: int=100):
        
        best_score = 0
        
        for i in range(generations):
            
            curr_score = self.score(self._individs)    
            count_individs = len(self._individs) 
            
            curr_best_score = curr_score.max()
            if curr_best_score > best_score:
                best_score = curr_best_score
                best_individ = self._individs[np.argmax(curr_score)]
            
            #print(f"generation: {i}\tbest score:{curr_score.max()}, count individs: {count_individs}")
            
            if count_individs <= 2:
                break
            
            self._individs = self._individs[self.selection(curr_score, int(count_individs/2))]
            all_parents = list(zip(self._individs[:-1], self._individs[1:]))
            
            for parent in all_parents[:int(count_individs/4)]:
                self._individs = np.vstack([    self._individs, 
                                                self.evolution_k_point(parent[0], parent[1], np.random.randint(2, self.len))])
            
        return best_individ
               
            
            
            
        

In [18]:
genetic = Genetic(count_tasks, 1000, 0, len(params), params, difficult, timeh)
best = genetic.solution(500)
score = genetic.score(best)

output = open("output.txt", "w")
for i in best:
    output.write(str(i)+' ')
output.close()

print(f"best individ:\n{best}\nscore:{score}")

best individ:
[2 0 4 4 6 9 5 2 5 5 3 9 9 9 3 5 3 0 5 8 5 5 1 5 6 9 1 8 2 8 6 5 4 2 8 2 9
 4 5 2 1 0 9 8 6 6 6 3 5 9 1 3 1 9 1 8 2 2 2 7 4 5 4 9 0 8 9 8 8 1 3 0 2 4
 5 0 7 4 5 3 1 5 9 4 0 1 3 7 6 8 1 4 6 6 6 0 3 9 4 8 9 5 6 2 6 1 5 9 3 1 2
 3 7 9 9 6 9 9 0 4 6 1 5 0 9 8 3 8 0 6 4 5 3 7 7 1 1 6 6 9 5 4 2 1 1 2 2 2
 5 4 0 3 1 6 7 9 1 5 8 3 0 3 9 9 1 8 3 4 4 2 5 2 0 0 9 4 8 8 9 9 2 6 8 6 5
 4 4 6 8 3 8 2 0 8 0 4 4 1 8 6 5 5 1 2 7 5 4 2 8 9 9 8 3 3 7 8 4 3 9 3 0 7
 9 9 3 6 9 5 1 5 1 8 8 9 1 5 1 3 5 9 5 8 6 3 5 3 2 7 2 7 2 2 1 2 8 0 3 3 3
 4 0 7 3 4 4 4 5 8 5 0 8 7 1 2 7 0 5 9 9 5 8 0 0 0 4 4 1 0 9 1 8 9 6 4 7 9
 2 1 7 2 4 9 9 4 3 6 0 0 1 8 9 5 2 0 1 6 9 7 6 7 6 9 2 0 3 8 6 1 3 8 8 0 8
 6 8 6 2 1 0 5 4 3 5 6 2 5 3 2 8 0 5 4 2 2 4 3 9 3 9 3 4 0 2 6 3 5 0 6 2 8
 2 8 9 8 9 0 1 2 8 2 7 1 1 6 3 5 6 5 9 0 7 1 7 5 1 6 4 3 3 9 3 3 1 4 7 1 2
 1 6 7 6 8 7 1 6 1 9 4 4 8 0 0 9 0 4 0 7 3 5 3 4 7 1 0 1 6 8 1 6 4 4 3 9 5
 1 5 2 5 6 5 2 3 1 6 1 8 6 4 4 6 8 3 9 8 6 5 8 5 0 1 8 4 1 8 1 6 3 8 8 0 5
 9 9 8 2 3 